#  Project Unit Tests (.ipynb)

This notebook contains the unit tests for the financial analytics project, fulfilling Task 6.

**Instructions:**
1.  Place this notebook inside the `tests/` folder.
2.  Ensure all the project modules (`metrics.py`, `parallel.py`, `portfolio.py`, etc.) are in the parent directory.
3.  Run all cells (`Cell` > `Run All`) to execute the test suite.

## 1. Setup and Imports

In [7]:
!pip install polars
!pip install plotly

  Using cached plotly-6.3.1-py3-none-any.whl.metadata (8.5 kB)
Using cached plotly-6.3.1-py3-none-any.whl (9.8 MB)


In [ ]:
import unittest
import pandas as pd
import polars as pl
import numpy as np
import io
import sys
import os
import concurrent.futures
from pandas.testing import assert_frame_equal

In [8]:
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    print(f"Adding parent directory to system path: {parent_dir}")
    sys.path.append(parent_dir)

try:
    from metrics import compute_rolling_pandas, compute_rolling_polars, compute_drawdown
    from parallel import sequential_execution, parallel_execution
    from portfolio import process_portfolio_sequentially
except ImportError as e:
    print(f"Error: Could not import modules. {e}")
    print("Please make sure this notebook is in a 'tests' folder and the .py files are in the parent directory.")

## 2. Test Case Definitions

In [ ]:
class TestProjectAnalytics(unittest.TestCase):

    def setUp(self):
        """Set up small, predictable mock data for all tests."""
        
        #  Data for Rolling Metrics & Parallel Tests
        #manually reconstruct
        self.test_data_pd = pd.DataFrame({
            'timestamp': pd.to_datetime([
                '2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05',
                '2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05'
            ]),
            'symbol': ['A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B'],
            'price': [100, 102, 101, 103, 104, 50, 50, 51, 52, 51]
        }).set_index('timestamp')
        
        self.test_data_pl = pl.DataFrame({
            'timestamp': pd.to_datetime([
                '2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05',
                '2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05'
            ]),
            'symbol': ['A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B'],
            'price': [100, 102, 101, 103, 104, 50, 50, 51, 52, 51]
        }).sort('timestamp')
        
        # Data for Portfolio Aggregation Test
        self.mock_latest_data = {
            'TEST1': {'price': 100},
            'TEST2': {'price': 200}
        }
        self.mock_price_history = {
            'TEST1': pd.Series([90, 100, 80, 100]), # Max DD: (80-100)/100 = -0.2
            'TEST2': pd.Series([180, 200, 210, 190]) # Max DD: (190-210)/210 = -0.0952
        }

    def test_pandas_rolling_sma(self):
        """1. Validate correctness of rolling metrics (Pandas)."""
        df = compute_rolling_pandas(self.test_data_pd.copy(), window=3)
        
        # Check 'A' at 2023-01-03
        val_a = df[df['symbol'] == 'A']['sma_20'].iloc[2]
        expected_sma_a = (100 + 102 + 101) / 3
        self.assertAlmostEqual(val_a, expected_sma_a)
        
        # Check 'B' at 2023-01-04
        val_b = df[df['symbol'] == 'B']['sma_20'].iloc[3]
        expected_sma_b = (50 + 51 + 52) / 3
        self.assertAlmostEqual(val_b, expected_sma_b)

    def test_polars_rolling_sma_equivalence(self):
        """2. Test pandas vs polars outputs for equivalence."""
        df_pl = compute_rolling_polars(self.test_data_pl.clone(), window=3)
        
        # Check 'A' at 2023-01-03
        val_a_pl = df_pl.filter(pl.col('symbol') == 'A')['sma_20'][2]
        expected_sma_a = (100 + 102 + 101) / 3
        self.assertAlmostEqual(val_a_pl, expected_sma_a)
        
        # Check 'B' at 2023-01-04
        val_b_pl = df_pl.filter(pl.col('symbol') == 'B')['sma_20'][3]
        expected_sma_b = (50 + 51 + 52) / 3
        self.assertAlmostEqual(val_b_pl, expected_sma_b)

    def test_parallel_consistency(self):
        """3. Confirm threading and multiprocessing produce consistent results."""
        # Note: We use the *original* df here, as the parallel functions
        # are designed to handle the full, unsorted, multi-symbol DataFrame.
        df_seq = sequential_execution(self.test_data_pd.copy())
        df_thread = parallel_execution(self.test_data_pd.copy(), concurrent.futures.ThreadPoolExecutor)
        df_proc = parallel_execution(self.test_data_pd.copy(), concurrent.futures.ProcessPoolExecutor)
        
        # Use pandas' testing utility to compare DataFrames
        assert_frame_equal(df_seq, df_thread)
        assert_frame_equal(df_seq, df_proc)

    def test_portfolio_aggregation(self):
        """4. Ensure portfolio aggregation matches expected totals."""
        test_portfolio = {
          "name": "Test Portfolio",
          "positions": [
            {"symbol": "TEST1", "quantity": 10} # Value = 10 * 100 = 1000
          ],
          "sub_portfolios": [{
              "name": "Sub",
              "positions": [
                {"symbol": "TEST2", "quantity": 5} # Value = 5 * 200 = 1000
              ],
              "sub_portfolios": []
          }]
        }
        
        result = process_portfolio_sequentially(
            test_portfolio, 
            self.mock_latest_data, 
            self.mock_price_history
        )
        
        # Test top-level aggregation
        self.assertAlmostEqual(result['total_value'], 2000.0) # 1000 + 1000
        
        # Test max drawdown (should be worst of all components)
        # TEST1 DD = -0.2, TEST2 DD = -0.0952
        self.assertAlmostEqual(result['max_drawdown'], -0.2)
        
        # Test sub-portfolio aggregation
        sub_portfolio_result = result['sub_portfolios'][0]
        self.assertAlmostEqual(sub_portfolio_result['total_value'], 1000.0)
        self.assertAlmostEqual(sub_portfolio_result['max_drawdown'], -0.0952, places=4)


## 3. Run Test Suite

In [ ]:
suite = unittest.TestSuite()
suite.addTest(unittest.makeSuite(TestProjectAnalytics))

runner = unittest.TextTestRunner(stream=io.StringIO(), verbosity=2)
result = runner.run(suite)

print("--- Unit Test Results ---")
print(result.stream.getvalue())

/var/folders/6y/cbtvkz6n68372_s73v5h09hr0000gn/T/ipykernel_23047/2928006571.py:3: DeprecationWarning: unittest.makeSuite() is deprecated and will be removed in Python 3.13. Please use unittest.TestLoader.loadTestsFromTestCase() instead.
  suite.addTest(unittest.makeSuite(TestProjectAnalytics))


--- Unit Test Results ---
test_pandas_rolling_sma (__main__.TestProjectAnalytics.test_pandas_rolling_sma)
1. Validate correctness of rolling metrics (Pandas). ... ok
test_parallel_consistency (__main__.TestProjectAnalytics.test_parallel_consistency)
3. Confirm threading and multiprocessing produce consistent results. ... ok
test_polars_rolling_sma_equivalence (__main__.TestProjectAnalytics.test_polars_rolling_sma_equivalence)
2. Test pandas vs polars outputs for equivalence. ... ok
test_portfolio_aggregation (__main__.TestProjectAnalytics.test_portfolio_aggregation)
4. Ensure portfolio aggregation matches expected totals. ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.444s

OK

